### PLEIADES Tutorial - SAMMY Exercise EX027: Generating Parameter Files Based on ENDF Files

This exercise demonstrates how to use SAMMY to generate parameter files starting from ENDF files. The goal is to familiarize analysts with the process of running SAMMY directly from ENDF data and modifying the output for further analysis.

#### Purpose
The purpose of this exercise is to learn how to:
1. Run SAMMY using ENDF files as input.
2. Generate SAMMY-style INPut and PARameter files for subsequent runs.
3. Modify and refine the generated files to fit experimental data.

#### Description
SAMMY can read File 2 of an ENDF file and perform a run as specified in the INPut file. The output includes SAMMY-style INPut and PARameter files, which can be used for further analysis. However, since ENDF files lack some information required by SAMMY, only simple no-Bayes runs can be performed directly from ENDF files.

#### Instructions
1. **Initial Run**:
    - Use the command file `ex027a` to set up a no-Bayes run starting from the ENDF file.
    - The INPut file `ex027a.inp` does not include spin group information, which will be extracted from the ENDF file.
    - The experimental data file `ex027a.dat` assumes a 10% uncertainty due to the lack of provided uncertainties in the original data.
    - Run this setup to generate SAMMY-style INPut and PARameter files.

2. **Refining the Output**:
    - Modify the generated INPut and PARameter files for further analysis:
      - Update the thickness to `0.002800 atoms/barn`.
      - Enable single-scattering corrections and normalize as cross sections.
    - For Bayesian analysis, flag at least one parameter in the PARameter file.
    - Adjust the uncertainty settings in the INPut file by setting the `VMIN` parameter in Card Set 7 to account for unrealistic weights on background points.

3. **Fitting the Data**:
    - Start with small energy ranges for better fits, as the data set contains many points and gaps.
    - Normalize the data appropriately. For example, in the energy range from 6 to 100 eV, a normalization factor of nearly 10 is required.
    - Experiment with fitting strategies to improve the fit quality.

This exercise provides a hands-on approach to understanding the workflow of using SAMMY with ENDF files and refining the results for meaningful analysis.


### PRE-SETUP: Initializing Directories and Files for the Example

To begin, we need to set up a temporary directory and link all the required SAMMY files to it. This ensures that the necessary files are organized and accessible for running the SAMMY exercise.

In this step, we will create symbolic links for the required files from the `ex027` example into the designated `working_dir`. These files are essential for configuring and executing the SAMMY run.

The directories are configured as follows:
- **Working Directory**: `/tmp/sammy_run`
- **Output Directory**: `/tmp/sammy_run/sammy_output`

The following files should be available in the `working_dir`:
- `ex027a.inp`: Input file defining the initial setup for the SAMMY run.
- `ex027a.par`: Parameter file generated from the ENDF data.
- `ex027a.dat`: Data file with experimental data assuming a 10% uncertainty.

Ensure that these files are correctly linked to the `working_dir` before proceeding with the analysis.

In [15]:
from pathlib import Path
import tempfile

# Set the working directory. Here we are going to use a temporary directory
temp_dir = tempfile.gettempdir()
working_dir = Path(temp_dir) / "sammy_endf"
endf_dir = Path(temp_dir) / "sammy_endf" / "endf"
output_dir = Path(working_dir) / "results"

# Create the directories if they do not exist
working_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)
endf_dir.mkdir(parents=True, exist_ok=True)

# Copy the ex012a.inp, ex012a.par, and ex012a.dat files from "../samexm/ex012/" to the working directory
# Define the source directory and the files to copy
source_dir = Path("../samexm/ex027/").resolve()
files_to_link = ["ex027.dat"]
source_endf_dir = source_dir / "endf"
endf_files_to_link = ["ex027a.par", "ex027a.inp"]

# Create symbolic links for each file in the working directory
for file_name in files_to_link:
    source_file = source_dir / file_name
    destination_file = working_dir / file_name
    if destination_file.exists():
        if destination_file.is_symlink():
            destination_file.unlink()  # Remove the existing symbolic link
            print(f"Removed existing symbolic link: {destination_file}")
        else:
            print(f"File already exists and is not a symbolic link: {destination_file}")
            continue  # Skip creating the link if it's not a symbolic link
    try:
        # Create the symbolic link
        destination_file.symlink_to(source_file)
    except FileNotFoundError as e:
        print(f"Error: Source file does not exist: {source_file}")
    except PermissionError as e:
        print(f"Error: Permission denied while creating symlink: {destination_file}")
    except Exception as e:
        print(f"Unexpected error while creating symlink: {e}")

# Create symbolic links for the ENDF files
for file_name in endf_files_to_link:
    source_file = source_endf_dir / file_name
    destination_file = endf_dir / file_name
    if destination_file.exists():
        if destination_file.is_symlink():
            destination_file.unlink()  # Remove the existing symbolic link
            print(f"Removed existing symbolic link: {destination_file}")
        else:
            print(f"File already exists and is not a symbolic link: {destination_file}")
            continue  # Skip creating the link if it's not a symbolic link
    try:
        # Create the symbolic link
        destination_file.symlink_to(source_file)
    except FileNotFoundError as e:
        print(f"Error: Source file does not exist: {source_file}")
    except PermissionError as e:
        print(f"Error: Permission denied while creating symlink: {destination_file}")
    except Exception as e:
        print(f"Unexpected error while creating symlink: {e}")


# Verify the files are in the working directory
print("Files in working directory:", list(working_dir.iterdir()))
print("ENDF files in ENDF directory:", list(endf_dir.iterdir()))

Removed existing symbolic link: /tmp/sammy_endf/ex027.dat
Unexpected error while creating symlink: [Errno 17] File exists: '/home/l280162/Programs/PLEIADES/examples/samexm/ex027/endf/ex027a.par' -> '/tmp/sammy_endf/endf/ex027a.par'
Removed existing symbolic link: /tmp/sammy_endf/endf/ex027a.inp
Files in working directory: [PosixPath('/tmp/sammy_endf/endf'), PosixPath('/tmp/sammy_endf/results'), PosixPath('/tmp/sammy_endf/ex027.dat')]
ENDF files in ENDF directory: [PosixPath('/tmp/sammy_endf/endf/ex027a.inp'), PosixPath('/tmp/sammy_endf/endf/ex027a.par')]


### Loading Relevant Modules

This section focuses on importing the necessary modules from PLEIADES and other libraries to configure and execute the SAMMY exercise. These modules are essential for managing SAMMY input/output files, setting up the working environment, and running SAMMY locally.

In [16]:
import subprocess

from pleiades.sammy.config import LocalSammyConfig              # Needed for configuring the SAMMY run directory 
from pleiades.sammy.backends.local import LocalSammyRunner      # Needed for running SAMMY locally
from pleiades.sammy.interface import SammyFiles                 # Needed for managing SAMMY input and output files 

### Configuring the SAMMY environment with PLEIADES

In this step, we set up the SAMMY environment by specifying the executable path, working directory, and output directory. The `LocalSammyConfig` class is used to create a configuration object that holds these details. This configuration will be used to manage the SAMMY runs effectively.

In [ ]:
# Grab the SAMMY executable 
sammy_executable = subprocess.run(["which", "sammy"], capture_output=True, text=True).stdout.strip()

# Set the working directory. Here we are going to use a temporary directory
working_dir = Path(temp_dir) / "sammy_endf"
endf_dir = Path(temp_dir) / "sammy_endf" / "endf"
output_dir = Path(working_dir) / "results"

# Create SAMMY configuration class
config = LocalSammyConfig(
    sammy_executable=sammy_executable,
    working_dir=working_dir,
    output_dir=output_dir
)

print(f"Using SAMMY executable: {config.sammy_executable}")
print(f"Using working directory: {config.working_dir}")
print(f"Using output directory: {config.output_dir}")

# Double check the success of configuration through PLEIADES validation
if config.validate():
    print("Configuration validated successfully.")
else:
    print("Configuration validation failed.")